In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


# Preparing data

Load the dataset

In [24]:
file_path = "C:/Users/Administrador/Desktop/Rami/Proyecto 5to/Datasets/Kaggle solar panel DS/spg.csv" 
data = pd.read_csv(file_path)
data

,temperature_2_m_above_gnd,relative_humidity_2_m_above_gnd,mean_sea_level_pressure_MSL,total_precipitation_sfc,snowfall_amount_sfc,total_cloud_cover_sfc,high_cloud_cover_high_cld_lay,medium_cloud_cover_mid_cld_lay,low_cloud_cover_low_cld_lay,shortwave_radiation_backwards_sfc,...,wind_direction_10_m_above_gnd,wind_speed_80_m_above_gnd,wind_direction_80_m_above_gnd,wind_speed_900_mb,wind_direction_900_mb,wind_gust_10_m_above_gnd,angle_of_incidence,zenith,azimuth,generated_power_kw
0,2.17,31,1035.0,0.0,0.0,0.0,0,0,0,0.00,...,312.71,9.36,22.62,6.62,337.62,24.48,58.753108,83.237322,128.33543,454.100950
1,2.31,27,1035.1,0.0,0.0,0.0,0,0,0,1.78,...,294.78,5.99,32.74,4.61,321.34,21.96,45.408585,75.143041,139.65530,1411.999400
2,3.65,33,1035.4,0.0,0.0,0.0,0,0,0,108.58,...,270.00,3.89,56.31,3.76,286.70,14.04,32.848282,68.820648,152.53769,2214.849300
3,5.82,30,1035.4,0.0,0.0,0.0,0,0,0,258.10,...,323.13,3.55,23.96,3.08,339.44,19.80,22.699288,64.883536,166.90159,2527.609200
4,7.73,27,1034.4,0.0,0.0,0.0,0,0,0,375.58,...,10.01,6.76,25.20,6.62,22.38,16.56,19.199908,63.795208,182.13526,2640.203400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4208,8.69,66,1025.1,0.0,0.0,100.0,100,100,100,257.21,...,285.42,25.50,283.06,24.01,282.99,25.20,22.700907,64.952098,167.06794,173.410560
4209,7.57,90,1026.1,0.0,0.0,100.0,79,100,100,210.04,...,6.91,13.00,4.76,11.61,7.13,10.44,19.307487,63.895118,182.28247,403.792220
4210,7.27,90,1026.3,0.1,0.0,100.0,73,100,100,113.92,...,2.29,8.71,352.87,7.95,354.81,22.32,25.249506,65.827032,197.33868,158.367780
4211,8.25,81,1025.5,0.0,0.0,100.0,74,66,100,186.90,...,11.31,6.29,13.24,5.94,14.04,15.48,36.361161,70.496313,211.30027,126.953060


Separate features and target 

In [9]:
X = data.drop(columns=["generated_power_kw"])
y = data["generated_power_kw"]

Normalize features

In [10]:
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

Split the dataset

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

Convert data to PyTorch tensors

In [12]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

Create DataLoader

In [108]:
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Starting the model

Define the neural network

In [169]:
class SolarPowerNN(nn.Module):
    def __init__(self, input_dim):
        super(SolarPowerNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 104)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 64)
        self.fc8 = nn.Linear(64, 32)
        self.fc9 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Initialize the model, loss function, and optimizer

In [225]:
input_dim = X_train.shape[1]
model = SolarPowerNN(input_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Training loop

In [236]:
num_epochs = 125
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
losses = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    for batch_X, batch_y in train_loader:
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        scheduler.step()
    
    avg_loss = epoch_loss / len(train_loader)
    losses.append(avg_loss)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}")

c:\Users\Administrador\Desktop\Rami\Proyecto 5to\prenv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 104])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Administrador\Desktop\Rami\Proyecto 5to\prenv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([42, 1])) that is different to the input size (torch.Size([42, 104])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [10/125], Loss: 26295.5323
Epoch [20/125], Loss: 26266.3016
Epoch [30/125], Loss: 26238.9461
Epoch [40/125], Loss: 26370.4297
Epoch [50/125], Loss: 26293.0243
Epoch [60/125], Loss: 26242.5388
Epoch [70/125], Loss: 26196.4709
Epoch [80/125], Loss: 26216.7890
Epoch [90/125], Loss: 26270.2148
Epoch [100/125], Loss: 26239.2472
Epoch [110/125], Loss: 26248.5228
Epoch [120/125], Loss: 26234.0080


Evaluate the model

In [246]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        test_loss += loss.item()

test_loss /= len(test_loader)
rmse = test_loss ** 0.5
print(f"Test RMSE: {rmse:.4f} kW")
print(f"Test Loss: {test_loss:.4f}")

Test RMSE: 393.6073 kW
Test Loss: 154926.6987


c:\Users\Administrador\Desktop\Rami\Proyecto 5to\prenv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 104])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\Administrador\Desktop\Rami\Proyecto 5to\prenv\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([11, 1])) that is different to the input size (torch.Size([11, 104])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


# Geting Predictions

In [255]:
new_data = np.array([
    [3.65, 33, 1035.4, 0.0, 0.0, 0.0, 0, 0, 0, 108.58, 0.0, 270.00, 3.89, 56.31, 3.76, 286.70, 14.04, 32.848282, 68.820648, 152.53769]
])

Ensure the new data has the correct dimensions

In [256]:
assert new_data.shape[1] == X_train.shape[1], "New data must have the same number of features as the training data."

Preprocess the new data using the same scaler

In [257]:
new_data_normalized = scaler.transform(new_data)

c:\Users\Administrador\Desktop\Rami\Proyecto 5to\prenv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Convert to PyTorch tensor

In [258]:
new_data_tensor = torch.tensor(new_data_normalized, dtype=torch.float32)

Set the model to evaluation mode

In [259]:
model.eval()

SolarPowerNN(
  (fc1): Linear(in_features=20, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=104, bias=True)
  (fc4): Linear(in_features=1024, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=256, bias=True)
  (fc6): Linear(in_features=256, out_features=128, bias=True)
  (fc7): Linear(in_features=128, out_features=64, bias=True)
  (fc8): Linear(in_features=64, out_features=32, bias=True)
  (fc9): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
)

Make predictions

In [260]:
with torch.no_grad():
    new_predictions = model(new_data_tensor)

Convert predictions to a readable format

In [261]:
new_predictions_np = new_predictions.numpy()

Display the predictions

In [262]:
predicted_power = new_predictions_np.flatten()
print("Predicted Power (kW):", new_predictions_np.flatten()[0])

Predicted Power (kW): 2396.2058
